# to get the permno of every components of s&p500 since 1960

In [15]:

import pandas as pd
import wrds

###################
# Connect to WRDS #
###################
conn=wrds.Connection()

WRDS recommends setting up a .pgpass file.
You can create this file yourself at any time
with the create_pgpass_file() function.
Loading library list...
Done


In [16]:
# msf500 = conn.raw_sql("""
#                         SELECT *
#                         FROM crsp.msf
#                         WHERE date > '12/31/2019'
#                         """)

In [17]:
# permno = conn.raw_sql("""
#                         SELECT a.*, b.date
#                         FROM crsp.dsp500list AS a,
#                         crsp.dsf AS b
#                         WHERE a.permno=b.permno
#                         AND b.date >= a.start and b.date <= a.ending
#                         AND b.date>'12/31/1959'
#                         ORDER BY date;
#                         """, date_cols=['start', 'ending', 'date'])

permno = conn.raw_sql("""
                        SELECT *
                        FROM crsp.dsp500list
                        WHERE ending > '12/31/1959'
                        """)

In [18]:
"""The WRDS-created linking dataset (ccmxpf_linktable) has been deprecated. It will continue to be created for a transition period of 1 year.
SAS programmers should use the Link History dataset (ccmxpf_lnkhist) from CRSP.  ---http://kaichen.work/?p=138"""
###################
#    CCM Block    #
###################
ccm = conn.raw_sql("""
                  select gvkey, lpermno as permno, linktype, linkprim, 
                  linkdt, linkenddt
                  from crsp.ccmxpf_lnkhist
                  where linktype in ('LU', 'LC')
                  """)

ccm['linkdt'] = pd.to_datetime(ccm['linkdt'])
ccm['linkenddt'] = pd.to_datetime(ccm['linkenddt'])

# if linkenddt is missing then set to today date
ccm['linkenddt'] = ccm['linkenddt'].fillna(pd.to_datetime('today'))

In [19]:
merged_df = permno.merge(ccm, how='left', on='permno')

In [30]:
permno_set = set(permno['permno'].apply(int).tolist())

In [21]:
permno_list = list(permno_set)

In [34]:
[value for value in permno_list if value <= 10000]

[]

In [22]:
len(permno_list)

1839

In [23]:
# create permno txt file
with open("s&p500_permno.txt", "w") as f:
    for value in permno_list:
        f.write(str(value) + '\n')
    f.close()

In [24]:
# create gvkey txt file
gvkey_list = merged_df['gvkey'].tolist()

In [25]:
gvkey_set = set(gvkey_list)

In [26]:
gvkey_list = list(gvkey_set)

In [27]:
gvkey_list = [x for x in gvkey_list if pd.isnull(x) == False]

In [28]:
with open("s&p500_gvkey.txt", "w") as f:
    for value in gvkey_list:
        f.write(str(value)+'\n')
    f.close()